# Symmetry between traders A and B: calculate Cost(B) in using the function for Cost(A)

In [22]:
from scipy.integrate import quad
def compute_sine_series_for_functions(a_func, b_func, kappa, lambd, N):
	"""
	Compute the sine series coefficients a_n and b_n for functions a_func and b_func.

	Parameters:
	a_func: function(t, lambd, kappa, N)
		The function a_func(t, lambd, kappa, N) to be expanded.
	b_func: function(t, lambd, kappa, N)
		The function b_func(t, lambd, kappa, N) to be expanded.
	lambd: float
		Parameter lambda.
	kappa: float
		Parameter kappa.
	N: int
		Number of terms in the series.

	Returns:
	a_coeffs: numpy array
		The sine series coefficients for a_func.
	b_coeffs: numpy array
		The sine series coefficients for b_func.
	"""
	pi = np.pi
	a_coeffs = np.zeros(N)
	b_coeffs = np.zeros(N)

	for n in range(1, N + 1):
		# Compute coefficients for a_func
		def integrand_a(t):
			return 2 * (a_func(t, kappa, lambd) - t) * np.sin(n * pi * t)

		coeff_a, _ = quad(integrand_a, 0, 1)
		a_coeffs[n - 1] = coeff_a

		# Compute coefficients for b_func
		def integrand_b(t):
			return 2 * (b_func(t, kappa, lambd) - t) * np.sin(n * pi * t)

		coeff_b, _ = quad(integrand_b, 0, 1)
		b_coeffs[n - 1] = coeff_b

	return a_coeffs, b_coeffs


def cost_fn_a_approx(a_n, b_n, kappa, lambd, verbose=False):
	"""
		This computes the cost function of trader A from the formula in the
		real-world constraints paper.
		This approach avoids computing integrals.

		:param a_n: Coefficients a_n for n from 1 to N
		:param b_n: Coefficients b_n for n from 1 to N
		:param kappa: Constant κ (kappa) - permanent market impact
		:param lambd: Constant λ (lambda) - temporary market impact

		:return: The computed value of the expression I
		"""
	# Ensure input sequences are numpy arrays
	a_n = np.array(a_n, dtype=np.float64)
	b_n = np.array(b_n, dtype=np.float64)
	n_coeffs = len(a_n)

	if len(b_n) != n_coeffs:
		raise ValueError("Sequences a_n and b_n must be of the same length.")

	pi = np.pi
	n = np.arange(1, n_coeffs + 1)

	# Calculate individual terms
	t1 = (2 + kappa) * (1 + lambd) / 2

	t2 = pi ** 2 / 2 * sum(i ** 2 * (a_n[i - 1] ** 2 + lambd * a_n[i - 1] * b_n[i - 1]) for i in n)

	t3 = 2 * kappa * lambd / pi * sum((b_n[i - 1] - a_n[i - 1]) / i for i in n if i % 2 == 1)

	t4 = 2 * kappa * sum((a_n[i - 1] + lambd * b_n[i - 1]) * a_n[j - 1] * i * j / (i * i - j * j)
	                     for i in n for j in n if (i + j) % 2 == 1)

	total_loss  = t1 + t2 + t3 + t4
	if verbose:
		print("APPROX TOTAL COST FUNCTION FROM APPROX FORMULA:")
		print("int_I: ", t1)
		print("int_II: ", t2)
		print("int_III: ", t3)
		print("int_IV: ", t4)

		print("Loss function approximation formula: ", total_loss)

	return total_loss


def cost_fn_b_approx(a_n, b_n, kappa, lambd, verbose=False):
	"""
		This computes the cost function of trader B from the formula in the
		real-world constraints paper.
		This approach avoids computing integrals.

		:param a_n: Coefficients a_n for n from 1 to N
		:param b_n: Coefficients b_n for n from 1 to N
		:param kappa: Constant κ (kappa) - permanent market impact
		:param lambd: Constant λ (lambda) - temporary market impact

		:return: The computed value of the expression I
		"""
	# Ensure input sequences are numpy arrays
	a_n = np.array(a_n, dtype=np.float64)
	b_n = np.array(b_n, dtype=np.float64)
	n_coeffs = len(a_n)

	if len(b_n) != n_coeffs:
		raise ValueError("Sequences a_n and b_n must be of the same length.")

	pi = np.pi
	n = np.arange(1, n_coeffs + 1)

	# Calculate individual terms
	t1 = (2 + kappa) * (1 + lambd) / 2

	t2 = pi ** 2 / 2 * sum(i ** 2 * (a_n[i - 1] * b_n[i - 1] + lambd * b_n[i - 1] ** 2) for i in n)

	t3 = 2 * kappa / pi * sum((a_n[i - 1] - b_n[i - 1]) / i for i in n if i % 2 == 1)

	t4 = 2 * kappa * sum((a_n[i - 1] + lambd * b_n[i - 1]) * b_n[j - 1] * i * j / (i * i - j * j)
	                     for i in n for j in n if (i + j) % 2 == 1)

	total_loss = lambd * (t1 + t2 + t3 + t4)
	if verbose:
		print("APPROX TOTAL COST FUNCTION FROM APPROX FORMULA:")
		print("int_I: ", t1)
		print("int_II: ", t2)
		print("int_III: ", t3)
		print("int_IV: ", t4)

		print("Loss function approximation formula: ", total_loss)

	return total_loss

In [28]:
import pytest as pt
import numpy as np

def test_cost_b_in_terms_of_cost_a(lambd, kappa, a_exp, b_exp):
    def a_func(t, kappa, lambd):
        return t ** a_exp + lambd * np.sin(np.pi * t)

    def b_func(t, kappa, lambd):
        return t ** b_exp + kappa * np.sin(2 * np.pi * t)

    def a_func_dot(t, kappa, lamb):
        return a_exp * t ** (a_exp - 1) + lambd * np.pi * np.cos(np.pi * t)

    def b_func_dot(t, kappa, lamb):
        return b_exp * t ** (b_exp - 1) + kappa * 2 * np.pi * np.cos(2 * np.pi * t)


    N = 10

    a_n, b_n = compute_sine_series_for_functions(a_func, b_func, kappa, lambd, N)

    # Exact Integral of the Fourier approximations
    I_approx_b = cost_fn_b_approx(a_n, b_n, kappa, lambd, verbose=False)
    I_approx_b1 = lambd**2 * cost_fn_a_approx(b_n, a_n, kappa, 1 / lambd, verbose=False)

    assert np.isclose(I_approx_b1, I_approx_b, atol=0.001)

In [40]:
# Test values: "lambd, kappa, a_exp, b_exp"
test_vals = [
    (1, 1, 1, 1),
    (10, 1, 1, 1),
    (5, 1, 0.5, 2),
    (10, 1, 0.5, 3),
    (50, 1, 3, 0.5),
    (100, 1, 2, 2),
    (1, 5, 0.2, 4),
    (1, 10, 4, 0.3),
    (1, 50, 0.5, 2),
    (2, 10, 4, 0.2),
    (10, 10, 0.2, 0.3),
    (100, 100, 4, 4),
]

print("Testing that C_b(x,y,lambda) == lamba^2 * C_a(y,x,1/lambda\n")
print("---------------------------------------------------------------")
for lambd, kappa, a_exp, b_exp in test_vals:
    test_cost_b_in_terms_of_cost_a(lambd, kappa, a_exp, b_exp)
    print(f"Values equal for lambd={lambd},\t kappa={kappa},\t a_exp={a_exp},\t b_exp={b_exp}")
print("\n---------------------\nAll tests passed")

Testing that C_b(x,y,lambda) == lamba^2 * C_a(y,x,1/lambda

---------------------------------------------------------------
Values equal for lambd=1,	 kappa=1,	 a_exp=1,	 b_exp=1
Values equal for lambd=10,	 kappa=1,	 a_exp=1,	 b_exp=1
Values equal for lambd=5,	 kappa=1,	 a_exp=0.5,	 b_exp=2
Values equal for lambd=10,	 kappa=1,	 a_exp=0.5,	 b_exp=3
Values equal for lambd=50,	 kappa=1,	 a_exp=3,	 b_exp=0.5
Values equal for lambd=100,	 kappa=1,	 a_exp=2,	 b_exp=2
Values equal for lambd=1,	 kappa=5,	 a_exp=0.2,	 b_exp=4
Values equal for lambd=1,	 kappa=10,	 a_exp=4,	 b_exp=0.3
Values equal for lambd=1,	 kappa=50,	 a_exp=0.5,	 b_exp=2
Values equal for lambd=2,	 kappa=10,	 a_exp=4,	 b_exp=0.2
Values equal for lambd=10,	 kappa=10,	 a_exp=0.2,	 b_exp=0.3
Values equal for lambd=100,	 kappa=100,	 a_exp=4,	 b_exp=4

---------------------
All tests passed
